In [31]:
# imports
import os

from autogen_agentchat.agents import AssistantAgent
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import dotenv
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from pathlib import Path

import httpx
import asyncio
import requests
from PIL import Image
from io import BytesIO
from pydantic import BaseModel, Field
from autogen_agentchat.base import TaskResult
from autogen_core.tools import FunctionTool
from autogen_agentchat.teams import RoundRobinGroupChat, Swarm, SelectorGroupChat
from autogen_core.models import UserMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent, CodeExecutorAgent
from autogen_core import Image as AGImage, CancellationToken  # We will use Image later
from autogen_agentchat.messages import TextMessage, MultiModalMessage, CodeExecutionEvent
from autogen_agentchat.ui import Console
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination, ExternalTermination
from autogen_ext.code_executors.docker import DockerCommandLineCodeExecutor

# Initialize

In [35]:
os.environ.pop('HTTP_PROXY', None)
os.environ.pop('HTTPS_PROXY', None)
os.environ.pop('http_proxy', None)
os.environ.pop('https_proxy', None)

dotenv.load_dotenv(override=True)
gemini_api_key = os.getenv("GEMINI_KEY")
open_router_api_key = os.getenv("OPENROUTER_KEY")

# Or set to empty
os.environ['HTTP_PROXY'] = ''
os.environ['HTTPS_PROXY'] = ''
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001",
    google_api_key = gemini_api_key,
)
print(embeddings.embed_query("testing!")[:5])

[-0.036367375, -0.019505447, 0.016166167, -0.057357196, -0.008019263]


In [36]:
##################
# Ollama Client. #
##################
ollama_client = OllamaChatCompletionClient(model="llama3.1:latest")

##########################################
# Deepseek free good for simple usecases #
##########################################
deepseek_client = OpenAIChatCompletionClient(
    base_url="https://openrouter.ai/api/v1",
    model="deepseek/deepseek-r1-0528:free",
    api_key=open_router_api_key,
    model_info={
        "family": "deepseek",
        "structured_output": False,
        "vision": True,
        "function_calling": True,
        "json_output": False
    },
    http_client=httpx.AsyncClient(trust_env=False)
)

###########################################
# Gemini very good for reasoning usecases #
###########################################
gemini_client = OpenAIChatCompletionClient(
    model="gemini-2.5-flash",
    api_key=gemini_api_key,
    http_client=httpx.AsyncClient(trust_env=False)
)

########################
# Testing model Client.#
########################
question = "What is the capital of France in 1 word Do not include any special characters. e.g. (Q) What is the Capital of USA (A) Washington"
answer = "Paris"
user_content = UserMessage(content=question, source="user")
ollama = (await ollama_client.create([user_content])).content[:5]
deepseek = (await deepseek_client.create([user_content])).content[:5]
gemini = (await gemini_client.create([user_content])).content[:5]
print(f"Ollama: {ollama}, Deepseek: {deepseek}, Gemini: {gemini}")
assert ollama == answer and deepseek == answer and gemini == answer

Ollama: Paris, Deepseek: Paris, Gemini: Paris


# Ingest to file

In [30]:
PDF_PATH="../project/A2013-18.pdf"
# 0. Verify PDF path
print("Current directory:", os.getcwd())
for file in os.listdir():
    print(f"  {file}")
if not os.path.exists(PDF_PATH):
    raise FileNotFoundError(f"PDF file not found: {PDF_PATH}")

# 1. Load PDF
print("Loading PDF...")
loader = PyPDFLoader(PDF_PATH)
documents = loader.load()
print(f"Loaded {len(documents)} pages")

# 2. Split into chunks
print("Splitting documents...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
chunks = text_splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

# 3. Create embeddings
print("Creating embeddings...")
embeddings = GoogleGenerativeAIEmbeddings(
    model="models/gemini-embedding-001"
)

# 4. Create and save FAISS vector store
print("Building FAISS index...")
vectorstore = FAISS.from_documents(
    documents=chunks,
    embedding=embeddings
)

# Save to disk as files
vectorstore.save_local("faiss_index")
print("✓ Vector store saved to 'faiss_index' folder")

# Later: Load from disk
print("\nLoading vector store from disk...")
loaded_vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True  # Required for pickle files
)

# Test query
query = "What is this document about?"
results = loaded_vectorstore.similarity_search(query, k=3)
print(f"\nQuery: {query}")
print(f"Top result:\n{results[0].page_content[:200]}")

Current directory: /Users/achuth.iyyatil/Code/personal/stunning-spork/notes
  Autogen Studio.pdf
  Source.zip
  Source
  Autogen Roadmap.excalidraw
  README.md
  Autogen Module 4.pdf
  Autogen .pdf
Loading PDF...
Loaded 370 pages
Splitting documents...
Created 1681 chunks
Creating embeddings...
Building FAISS index...
✓ Vector store saved to 'faiss_index' folder

Loading vector store from disk...

Query: What is this document about?
Top result:
Names, addresses, descriptions 
and occupations of subscribers 
Witnesses (along with names, addresses, 
descriptions and occupations) 
A.B. of………….Merchant Signed before me 
Signature……………. 
C.D. of…


In [57]:
async def fetch_law_points(query: str, k:int = 5) -> str:
    agent = AssistantAgent(
        name='summarizer_agent',
        model_client=gemini_client,
        system_message="""
        You are a summarization agent. Summarize the given text concisely.
        """,
    )
    results = loaded_vectorstore.similarity_search(query, k=k)
    combined_text = "\n\n".join([doc.page_content for doc in results])
    result = await agent.run(task=f"""
        Summarize the contents below based on the context provided.
        Context:
        {combined_text}
    """)
    return result.messages[-1].content
await fetch_law_points("What is this document about?")

"The provided text details the necessary information for subscribers and witnesses within a Memorandum of Association. For both subscribers and witnesses, it requires their names, addresses, descriptions, and occupations, along with their signatures. In the context of companies with share capital, it also specifies the number of shares taken by each subscriber. The document includes variations for one-person companies, where a nominee's details are also captured."

# The lawyers start convering with each other trying to counter one another with law points and a judge controls the conversation - use a selectoragent and run about 15 converesation loops

In [ ]:
import random

async def simulate_court_conversation(num_turns: int = 15, topic: str = "Breach of contract regarding delivery of goods."):
    # Initialize agents
    lawyer1 = AssistantAgent(
        name='Lawyer1',
        model_client=gemini_client,
        system_message="""
        You are Lawyer 1. Argue your case using relevant law points. Respond to the previous message and counter Lawyer 2.
        """,
    )
    lawyer2 = AssistantAgent(
        name='Lawyer2',
        model_client=gemini_client,
        system_message="""
        You are Lawyer 2. Argue your case using relevant law points. Respond to the previous message and counter Lawyer 1.
        """,
    )
    judge = AssistantAgent(
        name='Judge',
        model_client=gemini_client,
        system_message="""
        You are the Judge. Moderate the conversation, ask clarifying questions, and make decisions. Respond to both lawyers.
        """,
    )

    # Use the provided topic as the initial context
    context = topic
    conversation = [UserMessage(content=context, source="user")]

    for turn in range(num_turns):
        # Lawyers take turns
        current_lawyer = lawyer1 if turn % 2 == 0 else lawyer2
        prev_message = conversation[-1].content
        law_points = await fetch_law_points(prev_message)
        lawyer_prompt = f"Previous: {prev_message}\nRelevant Law Points: {law_points}\nYour argument?"
        lawyer_response = await current_lawyer.run(task=lawyer_prompt)
        conversation.append(UserMessage(content=lawyer_response.messages[-1].content, source=current_lawyer.name))

        # Judge responds every turn
        judge_prompt = f"Lawyer's statement: {lawyer_response.messages[-1].content}\nJudge, moderate or ask questions."
        judge_response = await judge.run(task=judge_prompt)
        conversation.append(UserMessage(content=judge_response.messages[-1].content, source="Judge"))

    # Print the conversation
    for msg in conversation:
        print(f"[{msg.source}]: {msg.content}\n")

# Run the simulation with a default topic, or pass your own
await simulate_court_conversation(num_turns=15, topic="""
Case Summary: Buy-Back Violation and Default in Deposit Repayment
TechVenture Solutions Ltd., a publicly listed company, decided to buy back 30% of its paid-up equity capital using proceeds from a previous share buyback conducted eight months earlier. The company proceeded without obtaining shareholder approval through a special resolution, arguing that board authorization was sufficient since the amount exceeded 10% of equity capital. Additionally, the company failed to extinguish the bought-back shares within the statutory timeframe and did not maintain proper records of the buyback transaction. During this period, TechVenture also accepted deposits from public members totaling ₹15 crores but failed to create the mandatory deposit repayment reserve account. When several deposits matured, the company defaulted on repayment, claiming financial difficulties due to market conditions.
The violations are severe and multifaceted. Under Section 68, the company breached multiple provisions: using proceeds from an earlier buyback of the same kind of shares (violating the proviso to sub-section 1), exceeding the 25% limit on buyback relative to paid-up capital and free reserves (sub-section 2(c)), failing to obtain special resolution approval for a buyback exceeding 10% (sub-section 2(b)), not extinguishing shares within seven days (sub-section 7), and failing to maintain a proper register of bought-back shares (sub-section 9). Under Section 73, the company violated the deposit acceptance provisions by failing to create a deposit repayment reserve account as required under clause (c) of sub-section (2), and subsequently defaulting on repayment obligations under sub-section (3). These violations expose the company to penalties ranging from ₹1 lakh to ₹3 lakh per violation under Section 68(11), with officers facing similar fines. More seriously, under Section 76A read with Section 74(3), the deposit default attracts imprisonment up to seven years for officers and fines between ₹25 lakhs to ₹2 crores, with company penalties ranging from ₹1 crore (or twice the deposit amount, whichever is lower) to ₹10 crores, demonstrating the legislature's intent to severely punish breaches of public trust in deposit-taking activities.
""")

# The lawyers start convering with each other trying to counter one another with law points and a judge controls the conversation - use a selectoragent and run about 15 converesation loops - with team

In [ ]:
import random

async def simulate_court_conversation_with_selector(num_turns: int = 15, topic: str = "Breach of contract regarding delivery of goods."):
    # Initialize agents
    lawyer1 = AssistantAgent(
        name='Lawyer1',
        model_client=gemini_client,
        system_message="""
        You are Lawyer 1. Argue your case using relevant law points. Respond to the previous message and counter Lawyer 2.
        Use the provided law points to strengthen your argument.
        """,
    )
    lawyer2 = AssistantAgent(
        name='Lawyer2',
        model_client=gemini_client,
        system_message="""
        You are Lawyer 2. Argue your case using relevant law points. Respond to the previous message and counter Lawyer 1.
        Use the provided law points to strengthen your argument.
        """,
    )
    judge = AssistantAgent(
        name='Judge',
        model_client=gemini_client,
        system_message="""
        You are the Judge. Moderate the conversation, select which lawyer should speak next, and ask clarifying questions or make decisions as needed.
        """,
    )

    # SelectorGroupChat with Judge as selector
    group_chat = SelectorGroupChat(
        participants=[lawyer1, lawyer2],
        selector=judge
    )

    # Initial message
    initial_message = UserMessage(content=topic, source="user")

    # Use run_stream to see intermediate outputs
    async for event in group_chat.run_stream(
        messages=[initial_message],
        max_rounds=num_turns
    ):
        # Print each event as it happens
        if hasattr(event, 'content') and hasattr(event, 'source'):
            print(f"[{event.source}]: {event.content}\n")

# Example usage:
await simulate_court_conversation_with_selector(num_turns=15, topic="Breach of contract regarding delivery of goods.")

# With selector group chat as team

In [68]:
async def simulate_selector_groupchat_law_conversation(
    num_turns: int = 15,
    topic: str = "Breach of contract regarding delivery of goods."
):
    # Agents
    lawyer1 = AssistantAgent(
        name='Lawyer1',
        model_client=gemini_client,
        system_message="""
        You are Lawyer 1. Argue your case using relevant law points. Respond to the previous message and counter Lawyer 2.
        Use the provided law points to strengthen your argument.
        """,
    )
    lawyer2 = AssistantAgent(
        name='Lawyer2',
        model_client=gemini_client,
        system_message="""
        You are Lawyer 2. Argue your case using relevant law points. Respond to the previous message and counter Lawyer 1.
        Use the provided law points to strengthen your argument.
        """,
    )
    judge = AssistantAgent(
        name='Judge',
        model_client=gemini_client,
        system_message="""
        You are the Judge. Moderate the conversation, select which lawyer should speak next, and ask clarifying questions or make decisions as needed.
        """,
    )

    # Termination condition: max turns
    termination_condition = MaxMessageTermination(max_messages=num_turns)

    # SelectorGroupChat as the team
    team = SelectorGroupChat(
        participants=[lawyer1, lawyer2, judge],
        model_client=gemini_client,
        termination_condition=termination_condition,
        max_turns=num_turns
    )

    # Use run_stream to see intermediate outputs
    async for event in team.run_stream(task=topic):
        # If it's a TextMessage from a lawyer, fetch law points
        if isinstance(event, TextMessage) and event.source in ['Lawyer1', 'Lawyer2']:
            # Assuming you have a fetch_law_points function defined
            # law_points = await fetch_law_points(event.content)
            # print(f"[Law Points for {event.source}]: {law_points}\n")
            print(f"[{event.source}]: {event.content}\n")
        # Print text messages
        elif isinstance(event, TextMessage):
            print(f"[{event.source}]: {event.content}\n")
        # Handle TaskResult (final result)
        elif isinstance(event, TaskResult):
            print(f"Conversation completed. Stop reason: {event.stop_reason}\n")
        else:
            # Print other event types for debugging
            print(f"[Event]: {type(event).__name__}\n")

[user]: Breach of contract regarding delivery of goods.

[Lawyer1]: Good morning, Your Honor, and to my learned colleague. I am Lawyer 1, representing the aggrieved party, [Client Name], in this matter concerning a clear and undeniable breach of contract.

Our position is straightforward: the defendant, [Defendant's Name, represented by Lawyer 2], has committed a fundamental breach of contract by failing to deliver the goods as stipulated in the legally binding agreement between our parties.

The contract, executed on [Date], clearly outlined the specific goods to be delivered, the quantity, quality, and crucially, the agreed-upon delivery date of [Specific Delivery Date]. My client, in good faith, fulfilled their part of the bargain by tendering payment in advance, or by being ready and willing to accept and pay for the goods upon delivery.

However, the defendant has failed entirely, or substantially failed, to meet their contractual obligations. This is not merely a minor deviation;

In [70]:
case = """**Case Summary: Buy-Back Violation and Default in Deposit Repayment**

TechVenture Solutions Ltd., a publicly listed company, decided to buy back 30% of its paid-up equity capital using proceeds from a previous share buyback conducted eight months earlier. The company proceeded without obtaining shareholder approval through a special resolution, arguing that board authorization was sufficient since the amount exceeded 10% of equity capital. Additionally, the company failed to extinguish the bought-back shares within the statutory timeframe and did not maintain proper records of the buyback transaction. During this period, TechVenture also accepted deposits from public members totaling ₹15 crores but failed to create the mandatory deposit repayment reserve account. When several deposits matured, the company defaulted on repayment, claiming financial difficulties due to market conditions.

The violations are severe and multifaceted. Under Section 68, the company breached multiple provisions: using proceeds from an earlier buyback of the same kind of shares (violating the proviso to sub-section 1), exceeding the 25% limit on buyback relative to paid-up capital and free reserves (sub-section 2(c)), failing to obtain special resolution approval for a buyback exceeding 10% (sub-section 2(b)), not extinguishing shares within seven days (sub-section 7), and failing to maintain a proper register of bought-back shares (sub-section 9). Under Section 73, the company violated the deposit acceptance provisions by failing to create a deposit repayment reserve account as required under clause (c) of sub-section (2), and subsequently defaulting on repayment obligations under sub-section (3). These violations expose the company to penalties ranging from ₹1 lakh to ₹3 lakh per violation under Section 68(11), with officers facing similar fines. More seriously, under Section 76A read with Section 74(3), the deposit default attracts imprisonment up to seven years for officers and fines between ₹25 lakhs to ₹2 crores, with company penalties ranging from ₹1 crore (or twice the deposit amount, whichever is lower) to ₹10 crores, demonstrating the legislature's intent to severely punish breaches of public trust in deposit-taking activities."""

await simulate_selector_groupchat_law_conversation(num_turns=15, topic=case)


[user]: **Case Summary: Buy-Back Violation and Default in Deposit Repayment**

TechVenture Solutions Ltd., a publicly listed company, decided to buy back 30% of its paid-up equity capital using proceeds from a previous share buyback conducted eight months earlier. The company proceeded without obtaining shareholder approval through a special resolution, arguing that board authorization was sufficient since the amount exceeded 10% of equity capital. Additionally, the company failed to extinguish the bought-back shares within the statutory timeframe and did not maintain proper records of the buyback transaction. During this period, TechVenture also accepted deposits from public members totaling ₹15 crores but failed to create the mandatory deposit repayment reserve account. When several deposits matured, the company defaulted on repayment, claiming financial difficulties due to market conditions.

The violations are severe and multifaceted. Under Section 68, the company breached multiple